In [ ]:
! jupyter comet_ml enable

In [1]:
#!/usr/bin/env python
# -*- coding:UTF-8 -*-

# from comet_ml import Experiment
import glob
import argparse
import os
import time
import tensorflow as tf
from model import RPN3D
from config import cfg
from utils import *
from utils.kitti_loader import iterate_data, sample_test_data
import subprocess



/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/mnt/storage/home/ja17618/.conda/envs/vox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
ctx = 'non_urban' 
tag = 'pre_trained_200'
output_path = 'non-urban_200' 
exp = 'non_urban'
bs=1
vis=False
GPU_AVAILABLE='1'
GPU_USE_COUNT=1


In [4]:
! nvidia-smi

Wed Aug  8 17:37:50 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.46                 Driver Version: 390.46                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:06:00.0 Off |                    0 |
| N/A   34C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [5]:
log = open('/mnt/storage/home/ja17618/vox_nurbgpu3.out','w+')
log.flush()

dataset_dir = cfg.DATA_DIR
val_dir = os.path.join(cfg.CONTEXT_DIR, ctx)
save_model_dir = os.path.join('./save_model', tag)
total_inference_time = []

# experiment = Experiment(api_key="xXtJguCo8yFdU7dpjEpo6YbHw",project_name=exp)
# create output folder

os.makedirs(output_path, exist_ok=True)
os.makedirs(os.path.join(output_path, 'data'), exist_ok=True)

if vis:
    os.makedirs(os.path.join(output_path, 'vis'), exist_ok=True)
process = subprocess.Popen('nvidia-smi -i 1 --format=csv -l 1 --query-gpu=index,timestamp,power.draw,utilization.gpu,clocks.current.sm,temperature.gpu,memory.used',stdout = log, shell=True)

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=cfg.GPU_MEMORY_FRACTION,
                        visible_device_list=GPU_AVAILABLE,
                        allow_growth=True)

    config = tf.ConfigProto(
        gpu_options=gpu_options,
        device_count={
            "GPU": GPU_USE_COUNT,
        },
        allow_soft_placement=True,
        log_device_placement=False
    )

    with tf.Session(config=config) as sess:
        model = RPN3D(
            cls=cfg.DETECT_OBJ,
            single_batch_size=bs,
            avail_gpus=GPU_AVAILABLE
        )
        if tf.train.get_checkpoint_state(save_model_dir):
            print("Reading model parameters from %s" % save_model_dir)
            model.saver.restore(
                sess, tf.train.latest_checkpoint(save_model_dir))
        counter=0
#         with experiment.test():
        for batch in iterate_data(val_dir, shuffle=False, aug=False, is_testset=False, batch_size=bs * GPU_USE_COUNT, multi_gpu_sum=GPU_USE_COUNT):
#             experiment.log_metric("counter",counter)

            if vis:
                tags, results, front_images, bird_views, heatmaps = model.predict_step(sess, batch, summary=False, vis=True)
            else:
                inference_start_time=time.time()
                tags, results = model.predict_step(sess, batch, summary=False, vis=False)
                inference_time = time.time()-inference_start_time
                total_inference_time.append(inference_time)

            # ret: A, B
            # A: (N) tag
            # B: (N, N') (class, x, y, z, h, w, l, rz, score)
            for tag, result in zip(tags, results):
                of_path = os.path.join(output_path, 'data', tag + '.txt')
                with open(of_path, 'w+') as f:
                    labels = box3d_to_label([result[:, 1:8]], [result[:, 0]], [result[:, -1]], coordinate='lidar')[0]
                    for line in labels:
                        f.write(line)
#                     print('write out {} objects to {}'.format(len(labels), tag))
            # dump visualizations
            if vis:
                for tag, front_image, bird_view, heatmap in zip(tags, front_images, bird_views, heatmaps):
                    front_img_path = os.path.join( output_path, 'vis', tag + '_front.jpg'  )
                    bird_view_path = os.path.join( output_path, 'vis', tag + '_bv.jpg'  )
                    heatmap_path = os.path.join( output_path, 'vis', tag + '_heatmap.jpg'  )
                    cv2.imwrite( front_img_path, front_image )
                    cv2.imwrite( bird_view_path, bird_view )
                    cv2.imwrite( heatmap_path, heatmap )
            counter +=1
process.kill()
log.flush()
log.close()
print('done')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Summary name Variable:0 is illegal; using Variable_0 instead.
INFO:tensorflow:Summary name Variable_1:0 is illegal; using Variable_1_0 instead.
INFO:tensorflow:Summary name Variable_2:0 is illegal; using Variable_2_0 instead.
INFO:tensorflow:Summary name VFE-1/kernel:0 is illegal; using VFE-1/kernel_0 instead.
INFO:tensorflow:Summary name VFE-1/bias:0 is illegal; using VFE-1/bias_0 instead.
INFO:tensorflow:Summary name VFE-1/gamma:0 is illegal; using VFE-1/gamma_0 instead.
INFO:tensorflow:Summary name VFE-1/beta:0 is illegal; using VFE-1/beta_0 instead.
INFO:tensorflow:Summary name VFE-1/moving_mean:0 is illegal; using VFE-1/moving_mean_0 instead.
INFO:tensorflow:Summary name VFE-1/moving_variance:0 is illegal; using VFE-1/moving_variance_0 instead.
INFO:tensorflow:Summary name VFE-2/kernel:0 is illegal; using VFE-2/kernel_0 instead.
INFO:tensorflow:Summary name VFE-2/bias:0 is illegal; using VFE-2

In [7]:
mean_inference = np.mean(total_inference_time)
min_inference = np.min(total_inference_time)
max_inference =np.max(total_inference_time)
var_inference = np.var(total_inference_time)

print('Min: %fs Max: %fs Mean: %fs Var: %fs ' %( min_inference,max_inference,mean_inference,var_inference))

Min: 0.112635s Max: 2.260123s Mean: 0.127845s Var: 0.004609s 
